# Qrack Clifford+RZ and Mitiq for High-Width RCS

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In the example below, we use Qiskit's `QuantumCircuit`, which makes this easier to combine with Mitiq, for further development.

In [1]:
width = 36
depth = 6
max_magic = 6

In [2]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

import os
import collections
import math
import multiprocessing
import numpy as np
import time
import random

from pyqrack import QrackSimulator, Pauli
from qiskit.circuit.quantumcircuit import QuantumCircuit

# from mitiq import zne
# from mitiq.zne.scaling.folding import fold_global
# from mitiq.zne.inference import RichardsonFactory

# Change these according to your OpenCL devices and system specifications:
os.environ['QRACK_MAX_ALLOC_MB']='15872'
os.environ['QRACK_MAX_PAGE_QB']='-1'
os.environ['QRACK_MAX_PAGING_QB']='-1'
os.environ['QRACK_MAX_CPU_QB']='-1'
# os.environ['QRACK_QUNITMULTI_DEVICES']='1'
# os.environ['QRACK_QPAGER_DEVICES']='1'
# os.environ['QRACK_QTENSORNETWORK_THRESHOLD_QB']='-1'
os.environ['QRACK_NONCLIFFORD_ROUNDING_THRESHOLD']='2'

2024-02-21 11:47:21.864025: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def cx(circ, q1, q2):
    circ.cx(q1, q2)

def cy(circ, q1, q2):
    circ.cy(q1, q2)

def cz(circ, q1, q2):
    circ.cz(q1, q2)

def acx(circ, q1, q2):
    circ.x(q1)
    circ.cx(q1, q2)
    circ.x(q1)

def acy(circ, q1, q2):
    circ.x(q1)
    circ.cy(q1, q2)

def acz(circ, q1, q2):
    circ.x(q1)
    circ.cz(q1, q2)
    circ.x(q1)

def swap(circ, q1, q2):
    circ.swap(q1, q2)

def nswap(circ, q1, q2):
    circ.cz(q1, q2)
    circ.swap(q1, q2)
    circ.cz(q1, q2)

def pswap(circ, q1, q2):
    circ.cz(q1, q2)
    circ.swap(q1, q2)   

def mswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.cz(q1, q2)

def iswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.cz(q1, q2)
    circ.s(q1)
    circ.s(q2)

def iiswap(circ, q1, q2):
    circ.sdg(q2)
    circ.sdg(q1)
    circ.cz(q1, q2)
    circ.swap(q1, q2)

def random_circuit(width, depth, magic):
    circuit = QuantumCircuit(width)

    magic_fraction = (3 * (width - 1) * depth / max_magic) if (width == 54) else (3 * width * depth / max_magic)
    
    # Nearest-neighbor couplers:
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    two_bit_gates = swap, pswap, mswap, nswap, iswap, iiswap, cx, cy, cz, acx, acy, acz

    col_len = math.floor(math.sqrt(width))
    while (((width // col_len) * col_len) != width):
        col_len -= 1
    row_len = width // col_len
    dead_qubit = 3

    for i in range(depth):
        # Single bit gates
        for j in range(width):
            if (width == 54) and (j == dead_qubit):
                continue

            for _ in range(3):
                # We're trying to cover 3 Pauli axes
                # with Euler angle axes x-z-x. 
                circuit.h(j)

                # We can trace out a quarter rotations around the Bloch sphere with stabilizer.
                rnd = random.randint(0, 3)
                if rnd & 1:
                    circuit.s(j)
                if rnd & 2:
                    circuit.z(j)

                # For each axis, there is a chance of "magic."
                if (magic > 0) and ((magic_fraction * random.random()) < 1):
                    angle = random.uniform(0, math.pi / 2)
                    circuit.rz(angle, j)
                    magic -= 1

        # Nearest-neighbor couplers:
        ############################
        gate = gateSequence.pop(0)
        gateSequence.append(gate)
        for row in range(1, row_len, 2):
            for col in range(col_len):
                temp_row = row
                temp_col = col
                temp_row = temp_row + (1 if (gate & 2) else -1);
                temp_col = temp_col + (1 if (gate & 1) else 0)

                if (temp_row < 0) or (temp_col < 0) or (temp_row >= row_len) or (temp_col >= row_len):
                    continue

                b1 = row * row_len + col
                b2 = temp_row * row_len + temp_col

                if (b1 >= width) or (b2 >= width):
                    continue

                g = random.choice(two_bit_gates)
                g(circuit, b1, b2)
    
    return circuit

Run this next cell several times to produce a noisy value, optionally tuning the width, depth, and magic settings above. (Then, we could theoretically mitigate with Mitiq.)

In [ ]:
%%time

from multiprocessing import Process

def run_in_parallel(task):
    running_tasks = [Process(target=task) for _ in range(os.cpu_count())]
    for running_task in running_tasks:
        running_task.start()
    for running_task in running_tasks:
        running_task.join()

def run_random_circuit():
    start = time.perf_counter()
    circ = random_circuit(width, depth, max_magic)
    qsim = QrackSimulator(circ.width(), isStabilizerHybrid=True, isTensorNetwork=False, isSchmidtDecompose=False)
    qsim.run_qiskit_circuit(circ, 0)
    # This is just to ensure that the non-Clifford buffers get flushed and rounded:
    qsim.prob_perm(list(range(width)), [False] * width)

    print("Fidelity: ", qsim.get_unitary_fidelity(), ", Time: ", time.perf_counter() - start, "s")

# run_in_parallel(run_random_circuit)
run_random_circuit()

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_2070_Super.ir
